In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-api-keys-here"

import json
from pathlib import Path
from openai import OpenAI
from ollama import Client as OllamaClient

# === CONFIGURATION ===
NUM_GENERATIONS = 3
PROMPT_FILE = "consent_seniors_prompts.json"  
OUTPUT_DIR = Path("outputs")
MODEL_A = "gpt-4o"
MODEL_B = "ollama_mistral"        
OLLAMA_MODEL_NAME = "mistral"    

# === SETUP ===
openai_client = OpenAI()
ollama_client = OllamaClient()
OUTPUT_DIR.mkdir(exist_ok=True)

with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    prompts = json.load(f)


def generate_with_gpt4o(prompt_text: str) -> str:
    """Call GPT-4o to generate a response."""
    chat = openai_client.chat.completions.create(
        model=MODEL_A,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant specialized in explaining digital consent "
                    "and privacy to older adults (65+). Use clear, plain language."
                ),
            },
            {"role": "user", "content": prompt_text},
        ],
        temperature=0.7,
        max_tokens=512,
    )
    return chat.choices[0].message.content.strip()


def generate_with_ollama(prompt_text: str) -> str:
    """Call Ollama local model to generate a response."""
    response = ollama_client.chat(
        model=OLLAMA_MODEL_NAME,
        messages=[
            {"role": "system",
             "content": (
                 "You are a helpful assistant specialized in explaining digital consent "
                 "and privacy to older adults (65+). Use clear, plain language."
             )},
            {"role": "user", "content": prompt_text},
        ],
    )
    # ollama's python client returns dict with 'message'
    return response["message"]["content"].strip()


results = []

for prompt in prompts:
    prompt_id = prompt["id"]
    prompt_text = prompt["prompt_text"]
    category = prompt.get("category", "")
    references = prompt.get("references", [])
    source_type = prompt.get("source_type", "")

    for i in range(NUM_GENERATIONS):
        gen_idx = i + 1

        print(f"Generating [GPT-4o] Prompt {prompt_id}, Iteration {gen_idx}")
        response_a = generate_with_gpt4o(prompt_text)
        results.append(
            {
                "prompt_id": prompt_id,
                "category": category,
                "references": references,
                "source_type": source_type,
                "model": MODEL_A,
                "gen_id": f"{prompt_id}_gpt4o_gen{gen_idx}",
                "response": response_a,
            }
        )

        print(f"Generating [Ollama] Prompt {prompt_id}, Iteration {gen_idx}")
        response_b = generate_with_ollama(prompt_text)
        results.append(
            {
                "prompt_id": prompt_id,
                "category": category,
                "references": references,
                "source_type": source_type,
                "model": MODEL_B,
                "gen_id": f"{prompt_id}_ollama_gen{gen_idx}",
                "response": response_b,
            }
        )

# save to jsonl
outfile = OUTPUT_DIR / "responses_seniors_gpt4o_vs_ollama.jsonl"
with open(outfile, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"✅ Done. Saved to {outfile}")


Generating [GPT-4o] Prompt SENIOR_P1, Iteration 1
Generating [Ollama] Prompt SENIOR_P1, Iteration 1
Generating [GPT-4o] Prompt SENIOR_P1, Iteration 2
Generating [Ollama] Prompt SENIOR_P1, Iteration 2
Generating [GPT-4o] Prompt SENIOR_P1, Iteration 3
Generating [Ollama] Prompt SENIOR_P1, Iteration 3
Generating [GPT-4o] Prompt SENIOR_P2, Iteration 1
Generating [Ollama] Prompt SENIOR_P2, Iteration 1
Generating [GPT-4o] Prompt SENIOR_P2, Iteration 2
Generating [Ollama] Prompt SENIOR_P2, Iteration 2
Generating [GPT-4o] Prompt SENIOR_P2, Iteration 3
Generating [Ollama] Prompt SENIOR_P2, Iteration 3
Generating [GPT-4o] Prompt SENIOR_P3, Iteration 1
Generating [Ollama] Prompt SENIOR_P3, Iteration 1
Generating [GPT-4o] Prompt SENIOR_P3, Iteration 2
Generating [Ollama] Prompt SENIOR_P3, Iteration 2
Generating [GPT-4o] Prompt SENIOR_P3, Iteration 3
Generating [Ollama] Prompt SENIOR_P3, Iteration 3
Generating [GPT-4o] Prompt SENIOR_P4, Iteration 1
Generating [Ollama] Prompt SENIOR_P4, Iteration 1
